In [ ]:
import openai
import os
import json

# read config file from root directory
config_path = '../config.json'
with open(config_path, 'r') as f:
    config = json.load(f)

# Set up OpenAI API credentials
os.environ["OPENAI_API_KEY"] = config['OPENAI_API_KEY']

# 1. Chat Format Introduction

## 1.1 Response returned by model:

In [ ]:
#### The chat completion object example
example = {
  "id": "chatcmpl-123",                       # A unique identifier for the chat completion.
  "object": "chat.completion",
  "created": 1677652288,
  "model": "gpt-3.5-turbo-0613",
  "choices": [{
    "index": 0,
    "message": {
      "role": "assistant",                    # The role of the author of this message.
      "content": "\n\nHello there, how may I assist you today?",
    },
    "finish_reason": "stop"                   # The reason why the chat stopped.
  }],
  "usage": {                                  # Usage statistics for the completion request.
    "prompt_tokens": 9,
    "completion_tokens": 12,
    "total_tokens": 21
  }
}

#### The chat completion chunk object example
# {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0613","choices":[{"index":0,"delta":{"role":"assistant","content":""},"finish_reason":null}]}

# {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0613","choices":[{"index":0,"delta":{"content":"Hello"},"finish_reason":null}]}

# {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0613","choices":[{"index":0,"delta":{"content":"!"},"finish_reason":null}]}

# ....

# {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0613","choices":[{"index":0,"delta":{"content":" today"},"finish_reason":null}]}

# {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0613","choices":[{"index":0,"delta":{"content":"?"},"finish_reason":null}]}

# {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0613","choices":[{"index":0,"delta":{},"finish_reason":"stop"}]}

## 1.2 Create chat completion (request and response):

In [ ]:
# No streaming
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-4",
  # temperature=0.9,  # between 0 and 2, higher -> more diversity
  # top_p=1,         # between 0 and 1, higher -> less diversity. Altering this or temperature but not both is recommended
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello! What's your model? gpt-3.5 or gpt-4?"},
  ]
)

# print(completion.choices[0].message)
completion

In [ ]:
# Streaming
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ],
  stream=True
)

for chunk in completion:
  print(chunk.choices[0].delta)

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

functions = [ 
  {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location", # used by the model to choose when and how to call the function.
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA",
            },
            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
    },
  }
]
messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]
completion = openai.ChatCompletion.create(
  model="gpt-4",
  messages=messages,
  functions=functions,
  function_call="auto",  # auto is default, but we'll be explicit. "none" means the model will not call a function and instead generates a message. "auto" means the model can pick between generating a message or calling a function. Specifying a particular function via {"name": "my_function"} forces the model to call that function
)

print(completion)
